In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import Dataset, DataLoader
from SVarM_utils import *
from SVarM_models import Classifier
from scipy.io import loadmat

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from torch.utils.data import Dataset, DataLoader
c=10

class MeshDataset(Dataset):
    def __init__(self, IDs):
        self.IDs = IDs

    def __len__(self):
        return len(self.IDs)

    def __getitem__(self, idx):
        file_path = self.IDs[idx]
        y = int(file_path.split("/")[1].split("_")[0])
        print(file_path)
        mesh = loadData(file_path.strip())  
        return mesh, torch.tensor([y])

matdata = loadmat("data_splits/MNIST_data_splits.mat")
test_data = matdata['test_data'].tolist()
train_data = matdata['train_data'].tolist()
val_dataset = MeshDataset(test_data)

In [3]:
model = Classifier(10)
sum(p.numel() for p in model.parameters() if p.requires_grad)
model.load_state_dict(torch.load("trained_models/mnist_classifier"))

<All keys matched successfully>

In [4]:
batch_size =12
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: list(zip(*x)))
criterion = nn.CrossEntropyLoss()

In [7]:
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for meshs, y in val_loader:
        print(y)
        y = torch.stack(y)
        outputs = model(meshs)
        print(y.shape,outputs.shape)
        loss = criterion(outputs, y)
        val_loss += loss.item()
        # Compute accuracy
        _, predicted = torch.max(outputs, 1)  # Get predicted class (index of max logit)
        total += y.size(0)  # Total number of samples
        correct += (predicted == y).sum().item()  # Count correct predictions
accuracy = 100 * correct / total 

val_loss /= len(val_loader)

print(f"Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

MNIST_remeshed/0_0.ply    
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_0.ply
MNIST_remeshed/0_1003.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1003.ply
MNIST_remeshed/0_1012.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1012.ply
MNIST_remeshed/0_1013.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1013.ply
MNIST_remeshed/0_1016.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1016.ply
MNIST_remeshed/0_1019.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1019.ply
MNIST_remeshed/0_1021.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1021.ply
MNIST_remeshed/0_1024.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1024.ply
MNIST_remeshed/0_1025.ply 
[Open3D WARNING] Read PLY failed: unable to open file: MNIST_remeshed\0_1025.ply
MNIST_remeshed/0_1026.ply 
[Ope

RuntimeError: Expected target size [12, 10], got [12, 1]